In [1]:
!pip install st-dbscan
!pip install pyproj
!pip install geopandas

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from st_dbscan import ST_DBSCAN
import datetime
from datetime import timedelta
from pyproj import Proj
import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point, LineString
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
discharges = pd.read_csv('drive/My Drive/project_ISA/data/discharges.csv', header=0, delimiter=',', index_col=0,
                         names=['date','longitude','latitude','polarity','magnitude','current'], parse_dates=['date'])
outages = pd.read_csv('drive/My Drive/project_ISA/data/outages.csv', header=0, delimiter=',', index_col=0,
                     names=['date','year','time','cause','outages_number','r_inf','r_sup'], parse_dates=['date'])
towers = pd.read_csv('drive/My Drive/project_ISA/data/towers.csv', header=0, delimiter=',', names=['longitude','latitude'])

In [5]:
def MAGNA_SIRGAS_projection(df):

    df_proj = gpd.GeoDataFrame(df,
                              geometry=gpd.points_from_xy(df.longitude,
                                                          df.latitude),
                              crs='EPSG:4326')
    df_proj = df_proj.to_crs('EPSG:3116')
    df_proj['x'], df_proj['y'] = df_proj['geometry'].x, df_proj['geometry'].y
    return df_proj

def buffer(df_proj, radious):
    buffer_dist = LineString(df_proj["geometry"]).buffer(radious)
    buffer_dist = gpd.GeoDataFrame(geometry=[buffer_dist], crs="EPSG:3116")#.to_crs(
#        "EPSG:4326"
#    )
    x_buffer, y_buffer = list(buffer_dist.iloc[0, 0].exterior.coords.xy)
    return x_buffer, y_buffer, buffer_dist

towers_proj = MAGNA_SIRGAS_projection(towers)
x_buffer, y_buffer, buffer_dist = buffer(towers_proj, 10000)
discharges_proj = MAGNA_SIRGAS_projection(discharges)

discharges_filter = discharges_proj.loc[discharges_proj.within(buffer_dist.geometry.iloc[0])]
discharges_filter
#towers_buffer = towers.iloc[1:240].reset_index(drop=True)

,date,longitude,latitude,polarity,magnitude,current,geometry,x,y
3,2018-04-01 00:02:17.110,-73.9832,6.6810,-1,12.7,-12.7,POINT (1010427.495 1230548.802),1.010427e+06,1.230549e+06
9,2018-04-01 00:03:59.230,-73.9841,6.7392,1,7.2,7.2,POINT (1010326.757 1236985.096),1.010327e+06,1.236985e+06
12,2018-04-01 00:08:40.553,-73.8479,7.1415,-1,11.0,-11.0,POINT (1025362.925 1281480.983),1.025363e+06,1.281481e+06
17,2018-04-01 00:09:25.713,-73.7494,6.8470,-1,7.6,-7.6,POINT (1036266.232 1248918.070),1.036266e+06,1.248918e+06
35,2018-04-01 00:13:14.373,-73.7124,6.9004,-1,6.4,-6.4,POINT (1040351.442 1254826.653),1.040351e+06,1.254827e+06
...,...,...,...,...,...,...,...,...,...
252057,2019-11-30 01:53:18.327,-73.8801,6.7491,1,4.3,4.3,POINT (1021824.117 1238083.364),1.021824e+06,1.238083e+06
252058,2019-11-30 01:53:18.560,-73.8331,6.6917,1,4.2,4.2,POINT (1027023.324 1231737.825),1.027023e+06,1.231738e+06
252059,2019-11-30 01:56:15.353,-73.9625,6.7108,1,6.0,6.0,POINT (1012715.497 1233844.858),1.012715e+06,1.233845e+06
252060,2019-11-30 01:56:15.353,-73.8714,6.7199,1,10.2,10.2,POINT (1022787.298 1234854.527),1.022787e+06,1.234855e+06


In [6]:
# Filters discharges that occured 24 h before each outage
def previous_discharges_by_time(outage_date):
    datetime_f = outage_date - timedelta(minutes=5)
    datetime_i = datetime_f - timedelta(hours=24)
    previous_discharges = discharges_filter[(discharges_filter['date']>=datetime_i) &
                                    (discharges_filter['date']<datetime_f)]
    return previous_discharges

discharges_24h = pd.DataFrame(columns=discharges.columns)

for outage_date in outages['date']:
    discharges_24h_outage = previous_discharges_by_time(outage_date=outage_date)
    discharges_24h = pd.concat([discharges_24h, discharges_24h_outage], axis=0)
        
discharges_24h

,date,longitude,latitude,polarity,magnitude,current,geometry,x,y
25223,2018-04-24 11:30:08.770,-73.7120,6.9432,1,13.1,13.1,POINT (1040392.011 1259560.050),1.040392e+06,1.259560e+06
25226,2018-04-24 20:39:34.133,-74.1211,6.4853,1,6.6,6.6,POINT (995178.198 1208905.768),9.951782e+05,1.208906e+06
25237,2018-04-24 21:44:36.783,-74.1096,6.4181,1,6.0,6.0,POINT (996449.767 1201474.139),9.964498e+05,1.201474e+06
25255,2018-04-24 21:53:15.967,-74.0527,6.6716,-1,6.1,-6.1,POINT (1002743.028 1229508.333),1.002743e+06,1.229508e+06
25256,2018-04-24 21:53:28.613,-73.8364,6.6774,-1,7.1,-7.1,POINT (1026659.227 1230156.204),1.026659e+06,1.230156e+06
...,...,...,...,...,...,...,...,...,...
246375,2019-11-15 02:41:39.450,-73.7393,6.7962,-1,23.8,-23.8,POINT (1037386.551 1243300.779),1.037387e+06,1.243301e+06
246376,2019-11-15 02:41:39.527,-73.7028,6.7866,-1,55.3,-55.3,POINT (1041422.249 1242242.066),1.041422e+06,1.242242e+06
246379,2019-11-15 02:42:18.277,-73.7869,6.7671,1,13.0,13.0,POINT (1032126.579 1240079.160),1.032127e+06,1.240079e+06
246381,2019-11-15 02:42:38.347,-73.7469,7.0671,1,6.0,6.0,POINT (1036525.551 1273259.640),1.036526e+06,1.273260e+06


In [7]:
df = discharges_24h[['date','x','y']].reset_index(drop=True)
df

,date,x,y
0,2018-04-24 11:30:08.770,1.040392e+06,1.259560e+06
1,2018-04-24 20:39:34.133,9.951782e+05,1.208906e+06
2,2018-04-24 21:44:36.783,9.964498e+05,1.201474e+06
3,2018-04-24 21:53:15.967,1.002743e+06,1.229508e+06
4,2018-04-24 21:53:28.613,1.026659e+06,1.230156e+06
...,...,...,...
18335,2019-11-15 02:41:39.450,1.037387e+06,1.243301e+06
18336,2019-11-15 02:41:39.527,1.041422e+06,1.242242e+06
18337,2019-11-15 02:42:18.277,1.032127e+06,1.240079e+06
18338,2019-11-15 02:42:38.347,1.036526e+06,1.273260e+06


In [8]:
date_min = datetime.datetime(2018,4,1,0)
date_max = datetime.datetime(2019,12,1,0)

def get_eps2(date_min, date_max, min_distance_minutes):
    max_days = (date_max-date_min).days
    min_distance_days = min_distance_minutes/60/24
    eps2 = min_distance_days*100/max_days
    return eps2

eps2 = get_eps2(date_min, date_max, 5)
eps2

0.0005701514322203977

In [9]:
xy_min = 970000
xy_max = 1500000

def get_eps1(xy_min, xy_max, min_distance_km):
    max_distance = xy_max-xy_min
    min_distance_meters = min_distance_km*1000
    eps1 = min_distance_meters*100/max_distance
    return eps1

eps1 = get_eps1(xy_min, xy_max, 3)
eps1

0.5660377358490566

In [10]:
df['x_norm'] = (df['x'] - xy_min) / (xy_max - xy_min)
df['y_norm'] = (df['y'] - xy_min) / (xy_max - xy_min)
df['date_norm'] = (df['date'] - date_min) / (date_max - date_min)
df

,date,x,y,x_norm,y_norm,date_norm
0,2018-04-24 11:30:08.770,1.040392e+06,1.259560e+06,0.132815,0.546340,0.038554
1,2018-04-24 20:39:34.133,9.951782e+05,1.208906e+06,0.047506,0.450766,0.039180
2,2018-04-24 21:44:36.783,9.964498e+05,1.201474e+06,0.049905,0.436744,0.039254
3,2018-04-24 21:53:15.967,1.002743e+06,1.229508e+06,0.061779,0.489638,0.039264
4,2018-04-24 21:53:28.613,1.026659e+06,1.230156e+06,0.106904,0.490861,0.039265
...,...,...,...,...,...,...
18335,2019-11-15 02:41:39.450,1.037387e+06,1.243301e+06,0.127144,0.515662,0.973912
18336,2019-11-15 02:41:39.527,1.041422e+06,1.242242e+06,0.134759,0.513664,0.973912
18337,2019-11-15 02:42:18.277,1.032127e+06,1.240079e+06,0.117220,0.509583,0.973912
18338,2019-11-15 02:42:38.347,1.036526e+06,1.273260e+06,0.125520,0.572188,0.973913


In [11]:
data = df.loc[:, ['date_norm','x_norm','y_norm']].values
len(data)

18340

In [ ]:
st_dbscan = ST_DBSCAN(eps1=eps1, eps2=eps2, min_samples=3) 
st_dbscan.fit(data[:])

In [14]:
data[:100].shape

(100, 3)